Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '../data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


**Reshape THE IMAGE TO 1-D vector.**

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


Build the computation graph.
    - We will build neural network with one hidden layer and 1024 neurons in that hidden layer.

In [161]:
batch_size = 128
relu_counts = 1024
graph = tf.Graph()
with graph.as_default():
#     Prepare the data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, relu_counts]))
    biases = tf.Variable(tf.zeros([relu_counts]))
    
    weights_two = tf.Variable(
        tf.truncated_normal([relu_counts, num_labels]))
    biases_two = tf.Variable(tf.zeros([num_labels]))
    
#     Construct relu neural network
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    tf_layer_two_input = tf.nn.relu(logits)
    
    logits_layer_two = tf.matmul(tf_layer_two_input, weights_two) + biases_two
    
#     Define Loss Function
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits_layer_two, tf_train_labels))
    
#     Optimize the Loss function. set learning rate as 0.5
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
#     Use the relu neural network
    train_prediction = tf.nn.softmax(logits_layer_two)
    
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights)+biases), weights_two)+biases_two)
    
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights)+biases), weights_two)+biases_two)

Before we runt the training. Firstly, we set up standard for measuring the performance of our model. The Accuracy, in other word.

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Run the computation and find out the accuracy of our neural network model.

In [163]:
num_steps = 3001

with(tf.Session(graph=graph)) as sess:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = sess.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%\n" % accuracy(
            valid_prediction.eval(), valid_labels))
    print()
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 25.670992
Minibatch accuracy: 10.2%
Validation accuracy: 13.6%

Minibatch loss at step 500: 1.486057
Minibatch accuracy: 80.5%
Validation accuracy: 60.6%

Minibatch loss at step 1000: 0.504380
Minibatch accuracy: 89.8%
Validation accuracy: 55.0%

Minibatch loss at step 1500: 0.238561
Minibatch accuracy: 94.5%
Validation accuracy: 54.5%

Minibatch loss at step 2000: 0.148781
Minibatch accuracy: 98.4%
Validation accuracy: 50.3%

Minibatch loss at step 2500: 0.034597
Minibatch accuracy: 99.2%
Validation accuracy: 51.1%

Minibatch loss at step 3000: 0.020692
Minibatch accuracy: 100.0%
Validation accuracy: 50.5%


Test accuracy: 55.5%


**Problem: How about we change the number of neurons in the hidden layer?**

Now we build a neural network with 2 hidden layers. 

In [22]:
batch_size = 128
hidden_layer_1_size = 300
hidden_layer_2_size = 200

graph = tf.Graph()
with graph.as_default():
#     Prepare the data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    hidden_layer_1_weights = tf.Variable(
        tf.truncated_normal([image_size*image_size, hidden_layer_1_size]))
    hidden_layer_1_biases = tf.Variable(tf.zeros([hidden_layer_1_size]))
    
    hidden_layer_1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_layer_1_weights) + hidden_layer_1_biases)
    
    hidden_layer_2_weights = tf.Variable(
        tf.truncated_normal([hidden_layer_1_size, hidden_layer_2_size]))
    hidden_layer_2_biases = tf.Variable(tf.zeros([hidden_layer_2_size]))
    hidden_layer_2 = tf.nn.relu(tf.matmul(hidden_layer_1, hidden_layer_2_weights) + hidden_layer_2_biases)
    
    output_weights = tf.Variable(
        tf.truncated_normal([hidden_layer_2_size, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(hidden_layer_2, output_weights) + output_biases
        
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    optimizer = tf.train.GradientDescentOptimizer(0.00001).minimize(loss)    
    train_prediction = tf.nn.softmax(logits)
    
    validation_hidden_layer_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_layer_1_weights) + hidden_layer_1_biases)
    validation_hidden_layer_2 = tf.nn.relu(tf.matmul(validation_hidden_layer_1, hidden_layer_2_weights) + hidden_layer_2_biases)

    validation_logits = tf.matmul(validation_hidden_layer_2, output_weights) + output_biases
    valid_prediction = tf.nn.softmax(validation_logits)

    
    test_hidden_layer_1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_layer_1_weights) + hidden_layer_1_biases)
    test_hidden_layer_2 = tf.nn.relu(tf.matmul(test_hidden_layer_1, hidden_layer_2_weights) + hidden_layer_2_biases)

    test_logits = tf.matmul(test_hidden_layer_2, output_weights) + output_biases
    test_prediction = tf.nn.softmax(test_logits)

In [25]:
num_steps = 20001

with(tf.Session(graph=graph)) as sess:
    tf.initialize_all_variables().run()
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        _, l, predictions = sess.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%\n" % accuracy(
            valid_prediction.eval(), valid_labels))
    print()
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Minibatch loss at step 0: 1479.637573
Minibatch accuracy: 8.6%
Validation accuracy: 10.8%

Minibatch loss at step 500: 827.644592
Minibatch accuracy: 16.4%
Validation accuracy: 16.3%

Minibatch loss at step 1000: 789.449829
Minibatch accuracy: 16.4%
Validation accuracy: 21.0%

Minibatch loss at step 1500: 529.038696
Minibatch accuracy: 23.4%
Validation accuracy: 28.5%

Minibatch loss at step 2000: 474.534515
Minibatch accuracy: 34.4%
Validation accuracy: 32.4%

Minibatch loss at step 2500: 440.469238
Minibatch accuracy: 43.0%
Validation accuracy: 37.1%

Minibatch loss at step 3000: 467.283600
Minibatch accuracy: 35.2%
Validation accuracy: 41.2%

Minibatch loss at step 3500: 335.178467
Minibatch accuracy: 45.3%
Validation accuracy: 44.7%

Minibatch loss at step 4000: 346.005554
Minibatch accuracy: 46.1%
Validation accuracy: 47.4%

Minibatch loss at step 4500: 355.823730
Minibatch accuracy: 40.6%
Validation accuracy: 49.5%

Minibatch loss at step 5000: 318.232056
Minibatch accuracy: 50.0